In [13]:
file_dir = '/fastdata/lip17adg/wandisco/talkback-FUSION-201811072236-dc3pl3332/fusion/logs/server/'
filename1 = 'gc.log.20181021_104556.0.current'
filename2 = 'gc.log.20181107_210001.0.current'
data = open(file_dir+filename1).read()
data += open(file_dir+filename2).read()

In [6]:
file_dir = '/fastdata/lip17adg/wandisco/talkback-FUSION-201811072236-dc3pl3402/fusion/logs/server/'
filename1 = 'gc.log.20181021_104630.0.current'
data = open(file_dir+filename1).read()

In [9]:
file_dir = '/fastdata/lip17adg/wandisco/talkback-FUSION-201811072237-gdcpl4172/fusion/logs/server/'
filename1 = 'gc.log.20180218_015625.0.current'
data = open(file_dir+filename1).read()

In [12]:
file_dir = '/fastdata/lip17adg/wandisco/talkback-FUSION-201811072237-gdcpl4173/fusion/logs/server/'
filename1 = 'gc.log.20180218_015640.0.current'
data = open(file_dir+filename1).read()

In [3]:
import re
import pandas as pd
import numpy as np

In [14]:
pattern = re.compile('(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{3})[\s\S]+?(\d.\d+)\ssecs[\s\S]+?(\d+.\d+)\]\s+\[GC\sWorker\sEnd[\s\S]+?(\d+.\d+[A-Z])\((\d+.\d+[A-Z])\)\-\>(\d+.\d+[A-Z])\((\d+.\d+[A-Z])\)[\s\S]+?(\d+.\d+[A-Z])\-\>(\d+.\d+[A-Z])[\s\S]+?(\d+.\d+[A-Z])\((\d+.\d+[A-Z])\)\-\>(\d+.\d+[A-Z])\((\d+.\d+[A-Z])\)[\s\S]+?user\=(\d+.\d+)[\s\S]+?sys\=(\d+.\d+)[\s\S]+?real\=(\d+.\d+)')    
info = pattern.findall(data)
columns = ['timestamp', 'GC pause', 'GC worker total', 'Eden1', 'Eden2', 'Eden3', 'Eden4', 'Survivors1', 'Survivors2', 'Heap_before_gc', 'Heap2', 'Heap_after_gc', 'Heap4', 'usertime', 'systime', 'realtime']
df= pd.DataFrame(info, columns=columns)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["systime"] = df["systime"].astype(np.float)
df["realtime"] = df["realtime"].astype(np.float)
df["usertime"] = df["usertime"].astype(np.float)
def sizeTransfer(size_info):
    symbol = size_info[-1]
    if symbol == 'G':
        return float(size_info[:-1]) * 1024
    elif symbol == 'B':
        return float(size_info[:-1]) / 1024 / 1024
    elif symbol == 'M':
        return float(size_info[:-1])

df["Heap_before_gc"] = df["Heap_before_gc"].map(sizeTransfer)
df["Heap_after_gc"] = df["Heap_after_gc"].map(sizeTransfer)
df["Eden1"] = df["Eden1"].map(sizeTransfer)
df["Eden2"] = df["Eden2"].map(sizeTransfer)
df["Eden3"] = df["Eden3"].map(sizeTransfer)
df["Eden4"] = df["Eden4"].map(sizeTransfer)
df["Survivors1"] = df["Survivors1"].map(sizeTransfer)
df["Survivors2"] = df["Survivors2"].map(sizeTransfer)
df["Heap2"] = df["Heap2"].map(sizeTransfer)
df["Heap4"] = df["Heap4"].map(sizeTransfer)
df['ratio'] = df['Heap_before_gc'] / df['Heap_after_gc']

In [15]:
df[["timestamp", "Heap_before_gc", "Heap_after_gc", "ratio"]][:10]

,timestamp,Heap_before_gc,Heap_after_gc,ratio
0,2018-10-21 10:46:13.019,384.0,53.4,7.191011
1,2018-10-21 10:46:13.068,389.0,75.2,5.172872
2,2018-10-21 10:46:15.481,2411.2,107.8,22.367347
3,2018-10-21 10:46:24.691,2411.8,123.3,19.560422
4,2018-10-21 10:46:26.241,2427.3,146.5,16.568601
5,2018-10-21 10:46:31.634,2418.5,146.0,16.565068
6,2018-10-21 10:46:35.259,2418.0,163.7,14.770922
7,2018-10-21 10:46:38.399,2403.7,205.6,11.691148
8,2018-10-21 10:46:42.409,2413.6,203.3,11.872110
9,2018-10-21 10:46:44.231,2411.3,170.1,14.175779


In [16]:
df[["timestamp", "Heap_before_gc", "Heap_after_gc", "ratio"]].to_csv("dc3pl3332.csv", index=False)